# User-wise operation

This notebook demostrates common user-wise operations by using SDK.

## Prerequisites

You need

- an SDK environment set up with the administrator priviledge. Check *environment setup.ipynb* for the detail.

**Replace the value below with your PrimeHub domain (*http* or *https*).**

In [ ]:
PRIMEHUB_CLUSTER = 'https://c.demo.primehub.io'

**Check if the configuration exists. If not, you need to login to retrieve the token and input it in the prompt. The configuration will be generated at**`~/.primehub/config.json`.

In [ ]:
import os
from primehub import PrimeHub, PrimeHubConfig

ph = PrimeHub(PrimeHubConfig())
if ph.is_ready():
    print(f"PrimeHub Python SDK {ph.version.version()} environment is ready, you are good to go.")
    print("Current Group:", ph.primehub_config.current_group)
else:
    print("Failed to retrieve the information from PrimeHub cluster, please check the configuration.")

## ToC

- [Fundamental Operations (CRUD)](#fundamental)
- [Common Operations](#common)

## Fundamental Operations <a class="anchor" id="fundamental"></a>

### Create user with/without admin role

Method `ph.admin.users.create(config)`

By using `create()` with a configuration to :
- create the user with the administration priviledge
- enable user personal volume with the specified 2Gb capacity
- Assign the user to the specified group

Before creating a user, let's decide which group the user belongs to. Check your PrimeHub and replace the `group_name` with yours.

In [ ]:
# Get the group id by the given group name
group_name = 'Continental'
groups = ph.admin.groups.list()
for group in groups:
    if group['name'] == group_name:
        assigned_group = group
print(assigned_group)

Let's prepare the configuration of creating a user, jonh-wick, and assign him to the group, continental.

In [ ]:
# Prepare a user configuration with admin role
config = {
  "username": "john-wick",
  "groups": {
    "connect": [ # assigned groups
      {
        "id": assigned_group['id'] # it takes group id only
      }
    ]
  },
  "isAdmin": True, # grant the user administration priviledge
  "volumeCapacity": 2 # user personal volume 
}

In [ ]:
data = ph.admin.users.create(config)
data

### Get User Info

Get the specified user information by the user's id.

In [ ]:
# Get the user id by the given user name
user_name = 'john-wick'
users = ph.admin.users.list()
for user in users:
    if user['username'] == user_name:
        specified_user = user

In [ ]:
# get the user info
ph.admin.users.get(specified_user['id'])

### Assign User One Group

Method `ph.admin.users.add_group(user_id, group_id`)

In [ ]:
# Get the user id by the given user name
user_name = 'john-wick'
users = ph.admin.users.list()
for user in users:
    if user['username'] == user_name:
        specified_user = user
specified_user

In [ ]:
# add_group(user_id, group_id)
ph.admin.users.add_group(specified_user['id'], assigned_group['id'])

# verify
ph.admin.users.list_groups(specified_user['id'])

### Remove User From One Group

Method `ph.admin.users.remove_group(user_id, group_id)`

In [ ]:
# remove John Wick from Continental
ph.admin.users.remove_group(specified_user['id'], assigned_group['id'])

# verify
ph.admin.users.list_groups(specified_user['id'])

### Update User

Method `ph.admin.users.update(user_id, config)`

By using `update()`, we can have a user sophiscated updates at once

Such as two changes as below in one update
- remove the user from the group
- revoke user's administration prviledge

In [ ]:
# Get the user id by the given user name
user_name = 'john-wick'
users = ph.admin.users.list()
for user in users:
    if user['username'] == user_name:
        specified_user = user
specified_user

Let's remove john-wick from *Continental* and revoke his administration priviledge (*Excommunicado*) in one update.

In [ ]:
config = {
  "groups": {
    "disconnect": [ # assigned groups
      {
        "id": assigned_group['id']
      }
    ]
  },
  "isAdmin": False, # grant the user administration priviledge
}

In [ ]:
result = ph.admin.users.update(specified_user['id'], config) # it takes the specified user id only.
result

### Delete User

Delete the specified user by the user's id.

In [ ]:
# Get the user id by the given user name
user_name = 'john-wick'
users = ph.admin.users.list()
for user in users:
    if user['username'] == user_name:
        specified_user = user
specified_user

In [ ]:
ph.admin.users.delete(specified_user['id'])

## Common Operations <a class="anchor" id="common"></a>

### Get group list of a specified user

In [ ]:
# Get the user id by the given user name
user_name = 'john-wick'
users = ph.admin.users.list()
for user in users:
    if user['username'] == user_name:
        specified_user = user
specified_user

In [ ]:
ph.admin.users.get(specified_user['id'])['groups']

### Get admin user list

In [ ]:
users = ph.admin.users.list()
admin_users = []
for user in users:
    if user['isAdmin']:
        admin_users.append(user)
admin_users

### Get disabled user list

In [ ]:
users = ph.admin.users.list()
disabled_users = []
for user in users:
    if not user['enabled']:
        disabled_users.append(user)
disabled_users